In [ ]:
import numpy as np

## 付録E　電気ヒートポンプ給湯機

### E.1 記号及び単位

省略

### E.2 消費電力量

1日当たりの沸き上げ時間$ \Delta t_{wh} $は 8 時間とする。

$$\large
E_{E,hs,d,t} = \left\{ \begin{array}{ll}
    (f_{ex} \times \theta_{ex,NAve,d} + f_{L} \times L''_{d} + f_{e,rtd} \times e_{rtd} + f_{itc})\\
    \large \times (1 + f_{mode2} \times r_{mode2}) \times C_{LT,d} \times C_{def,d} \times 10^{3} \div 3600 \div \Delta t_{wh} & \big(t=0～6 \big)\\ 
    0 & \big(t=7～22 \big)\\ 
    (f_{ex} \times \theta_{ex,NAve,d+1} + f_{L} \times L''_{d+1} + f_{e,rtd} \times e_{rtd} + f_{itc})\\
    \large \times (1 + f_{mode2} \times r_{mode2}) \times C_{LT,d+1} \times C_{def,d+1} \times 10^{3} \div 3600 \div \Delta t_{wh} & \big(t=23 \big)\\ 
    \end{array}
\right.
$$

$C_{def,d}$：日付$d$における1 日当たりのデフロスト運転による補正係数(-)  
$C_{LT,d}$：日付$d$における1 日当たりの低外気温領域でのヒートポンプ効率低下による補正係数(-)  
$E_{E,hs,d,t}$：日付$d$の時刻$t$における1 日当たりの給湯機の消費電力量(kWh/h)  
$e_{rtd}$：当該給湯機の効率(-)  
$L''_{d}$：日付$d$における1 日当たりの太陽熱補正給湯熱負荷(MJ/d)  
$r_{mode2}$：制御におけるセカンドモードの割合(-)  
$\Delta t_{wh}$：1日当たりの沸き上げ時間(h/d)  
$\theta_{ex,NAve,d}$：日付$d$における夜間平均外気温度(℃)  
$f_{ex}, f_{L}, f_{e,rtd}, f_{itc}, f_{mode2}$：係数(-)  


In [ ]:
# 1日当たりの給湯機の消費電力量 (1)
def get_E_E_hs_d_t(L_dashdash_k_d_t, L_dashdash_s_d_t, L_dashdash_w_d_t, L_dashdash_b1_d_t, L_dashdash_b2_d_t,
                   L_dashdash_ba1_d_t, L_dashdash_ba2_d_t,
                   e_rtd=None, m1see=None, Theta_ex_Nave=None):
    """
    :param L_dashdash_k_d_t: 1時間当たりの台所水栓における太陽熱補正給湯熱負荷 (MJ/h)
    :param L_dashdash_s_d_t: 1時間当たりの浴室シャワー水栓における太陽熱補正給湯負荷 (MJ/h)
    :param L_dashdash_w_d_t: 1時間当たりの洗面水栓における太陽熱補正給湯負荷 (MJ/h)
    :param L_dashdash_b1_d_t: 1時間当たりの浴槽水栓湯はり時における太陽熱補正給湯負荷 (MJ/h)
    :param L_dashdash_b2_d_t: 1時間当たりの浴槽自動湯はり時における太陽熱補正給湯負荷 (MJ/h)
    :param L_dashdash_ba1_d_t: 1時間当たりの浴槽水栓さし湯時における太陽熱補正給湯負荷 (MJ/h)
    :param L_dashdash_ba2_d_t: 1時間当たりの浴槽追焚時における太陽熱補正給湯負荷 (MJ/h)
    :param e_rtd: 当該給湯機の効率
    :param Theta_ex_Nave: 夜間平均外気温 (℃)
    :param m1see: M1スタンダードに基づくJIS相当効率
    :return: 1日当たりの給湯機の消費電力量 (kWh/d)
    """
    # 効率の決定
    if e_rtd is None:
        if m1see is None:
            e_rtd = get_e_rtd_default()
        else:
            e_rtd = m1see
    else:
        e_rtd = min(3.6, e_rtd)

    # 係数
    f_ex, f_L, f_e_rtd, f_itc, f_mode2 = get_coefficients()

    # 制御におけるセカンドモードの割合
    r_mode2 = get_r_mode2()

    # 低外気温領域でのヒートポンプ効率低下による補正係数
    C_LT_d = get_C_LT_d(Theta_ex_Nave)

    # デフロスト運転における補正係数
    C_def_d = get_C_def_d(Theta_ex_Nave)

    # 太陽熱補正給湯熱負荷
    L_dashdash_k_d = get_L_dashdash_k_d(L_dashdash_k_d_t)
    L_dashdash_s_d = get_L_dashdash_s_d(L_dashdash_s_d_t)
    L_dashdash_w_d = get_L_dashdash_w_d(L_dashdash_w_d_t)
    L_dashdash_b1_d = get_L_dashdash_b1_d(L_dashdash_b1_d_t)
    L_dashdash_b2_d = get_L_dashdash_b2_d(L_dashdash_b2_d_t)
    L_dashdash_ba1_d = get_L_dashdash_ba1_d(L_dashdash_ba1_d_t)
    L_dashdash_ba2_d = get_L_dashdash_ba2_d(L_dashdash_ba2_d_t)
    L_dashdash_d = get_L_dashdash_d(L_dashdash_k_d, L_dashdash_s_d, L_dashdash_w_d, L_dashdash_b1_d, L_dashdash_b2_d,
                                    L_dashdash_ba1_d, L_dashdash_ba2_d)

    # 24時間化
    Theta_ex_Nave = np.repeat(Theta_ex_Nave, 24)
    L_dashdash_d = np.repeat(L_dashdash_d, 24)
    C_LT_d = np.repeat(C_LT_d, 24)
    C_def_d = np.repeat(C_def_d, 24)

    # 1日後
    Theta_ex_Nave_d_p1 = np.roll(Theta_ex_Nave, -24)
    L_dashdash_d_p1 = np.roll(L_dashdash_d, -24)
    C_LT_d_p1 = np.roll(C_LT_d, -24)
    C_def_d_p1 = np.roll(C_def_d, -24)

    E_E_hs_d_t = np.zeros(24 * 365)
    t_wh = 8

    # (1) t == 0 - 6
    f1 = np.tile(np.arange(24) <= 6, 365)
    E_E_hs_d_t[f1] = (f_ex * Theta_ex_Nave[f1] + f_L * L_dashdash_d[f1] + f_e_rtd * e_rtd + f_itc) \
                     * (1 + f_mode2 * r_mode2) * C_LT_d[f1] * C_def_d[f1] * 10 ** 3 / 3600 / t_wh

    # (2) t == 7-22
    f2 = np.tile(np.logical_and(7 <= np.arange(24), np.arange(24) <= 22), 365)
    E_E_hs_d_t[f2] = 0

    # (3) t == 23
    f3 = np.tile(np.arange(24) == 23, 365)
    E_E_hs_d_t[f3] = (f_ex * Theta_ex_Nave_d_p1[f3] + f_L * L_dashdash_d_p1[f3] + f_e_rtd * e_rtd + f_itc) \
                     * (1 + f_mode2 * r_mode2) * C_LT_d_p1[f3] * C_def_d_p1[f3] * 10 ** 3 / 3600 / t_wh

    return E_E_hs_d_t

<div style="text-align: center;font-weight: bold;">表　係数</div>

<table>
<tr><th>$f_{ex}$</th><td>-0.302</td></tr>
<tr><th>$f_{L}$</th><td>0.263</td></tr>
<tr><th>$f_{e,rtd}$</th><td>-4.383</td></tr>
<tr><th>$f_{itc}$</th><td>20.754</td></tr>
<tr><th>$f_{mode2}$</th><td>0.235</td></tr>
</table>


In [ ]:
def get_coefficients():
    # 表E.3 係数
    table_e_3 = (
        -0.302,
        0.263,
        -4.383,
        20.754,
        0.235
    )
    return table_e_3

当該給湯機の効率$e_{rtd}$は、2.7 とするか、対象機器の JIS C 9220 に基づく年間給湯保温効率（JIS）、年間給湯効率（JIS）、M1 スタンダードに基づく JIS 相当効率を用いることができる。

In [ ]:
# 規定の当該給湯器の効率
def get_e_rtd_default():
    return 2.7

ふろ機能の種類が「ふろ給湯機（追焚あり）」の場合

$$\large
e_{rtd} = e_{APF} -  0.7 
$$

ふろ機能の種類が「給湯単機能」又は「ふろ給湯機（追焚なし）」の場合

$$\large
e_{rtd} = e_{APF} -  0.5 
$$

$e_{APF}$：年間給湯効率(-)  
$e_{rtd}$：当該給湯機の効率(-)  

換算した値は、3.6 を超える場合には 3.6 に等しいとする。

In [ ]:
# e_APFからe_rtdへの変換
def get_e_rtd(e_APF, bath_function):
    """
    :param e_APF: 日本冷凍空調工業 会標準規格 JRA4050:2007R に基づく年間給湯効率（APF）
    :param bath_function: ふろ機能の種類
    :return: 当該給湯器の効率
    """
    # e_APFからe_rtdへの変換
    if bath_function == 'ふろ給湯機(追焚あり)':
        e_rtd = e_APF - 0.7  # (2a)
    elif bath_function == '給湯単機能' or bath_function == 'ふろ給湯機(追焚なし)':
        e_rtd = e_APF - 0.5  # (2b)
    else:
        raise NotImplementedError()

    # 換算値が3.6を超える場合は3.6に等しいとする
    e_rtd = min(3.6, e_rtd)

    return e_rtd

制御におけるセカンドモードの割合$r_{mode2}$は0.4とする。 

In [ ]:
def get_r_mode2():
    return 0.4

$$\large
C_{LT,d} = \left\{ \begin{array}{ll}
    1.0 &(7 \leq \theta_{ex,Nave,d}の場合)\\ 
    1.0 + 0.0133 \times (7 - \theta_{ex,Nave,d}) &(\theta_{ex,Nave,d} < 7の場合)\\ 
    \end{array}
\right.
$$

$C_{LT,d}$：日付$d$における1 日当たりの低外気温領域でのヒートポンプ効率低下による補正係数(-)  
$\theta_{ex,NAve,d}$：日付$d$における夜間平均外気温度(℃)  

In [ ]:
# 1日当たりの低外気温領域でのヒートポンプ効率低下による補正係数 (3)
def get_C_LT_d(Theta_ex_Nave):
    """
    :param Theta_ex_Nave: 夜間平均外気温 (℃)
    :return: 1日当たりの低外気温領域でのヒートポンプ効率低下による補正係数
    """
    C_LT_d = np.ones(365)

    # Theta_ex_Nave < 7 の場合
    f = Theta_ex_Nave < 7
    C_LT_d[f] = 1.0 + 0.0133 * (7 - Theta_ex_Nave[f])

    return C_LT_d

$$\large
C_{def,d} = \left\{ \begin{array}{ll}
    1.0 &(5 \leq \theta_{ex,Nave,d}の場合)\\ 
    1.175 - \theta_{ex,Nave,d} \times 0.035 &(2 \leq \theta_{ex,Nave,d}<5の場合)\\ 
    1.12 - \theta_{ex,Nave,d} \times 0.0075 &(-2 \leq \theta_{ex,Nave,d}<2の場合)\\ 
    1.155 + \theta_{ex,Nave,d} \times 0.01 &(-15.5 \leq \theta_{ex,Nave,d}<-2の場合)\\ 
    1.0 &(\theta_{ex,Nave,d}<-15.5の場合)\\ 
    \end{array}
\right.
$$

$C_{def,d}$：日付$d$における1 日当たりのデフロスト運転による補正係数(-)  
$\theta_{ex,NAve,d}$：日付$d$における夜間平均外気温度(℃)  

In [ ]:
# 1日当たりのデフロスト運転における補正係数 (4)
def get_C_def_d(Theta_ex_Nave):
    """
    :param Theta_ex_Nave: 夜間平均外気温 (℃)
    :return: 1日当たりのデフロスト運転における補正係数
    """
    C_def_d = np.ones(365)

    # 2 <= Theta_ex_Nave < 5 の場合
    f1 = np.logical_and(2 <= Theta_ex_Nave, Theta_ex_Nave < 5)
    C_def_d[f1] = 1.175 - Theta_ex_Nave[f1] * 0.035

    # -2 <= Theta_ex_Nave < 2 の場合
    f2 = np.logical_and(-2 <= Theta_ex_Nave, Theta_ex_Nave < 2)
    C_def_d[f2] = 1.12 - Theta_ex_Nave[f2] * 0.0075

    # -15.5 <= Theta_ex_Nave < -2 の場合
    f3 = np.logical_and(-15.5 <= Theta_ex_Nave, Theta_ex_Nave < -2)
    C_def_d[f3] = 1.155 + Theta_ex_Nave[f3] * 0.01

    return C_def_d

### E.3 ガス消費量

日付$d$の時刻$t$における 1時間当たりの給湯機のガス消費量$E_{G,hs,d,t}$は0とする。

In [ ]:
# 1時間当たりの給湯機のガス消費量 (MJ/h)
def get_E_G_hs_d_t():
    # 1日当たりの給湯機のガス消費量は0とする
    return np.zeros(24 * 365)

### E.4 灯油消費量

日付$d$の時刻$t$における 1時間当たりの給湯機の灯油消費量$E_{K,hs,d,t}$は0とする。

In [ ]:
# 1時間当たりの給湯機の灯油消費量 (MJ/h)
def get_E_K_hs_d_t():
    # 1日当たりの給湯機の灯油消費量は0とする
    return np.zeros(24 * 365)

### E.5 １日当たりの太陽熱補正給湯熱負荷

$$\large
\displaystyle
L''_{d} = L''_{k,d}+L''_{s,d}+L''_{w,d}+L''_{b1,d}+L''_{b2,d}+L''_{ba1,d}+L''_{ba2,d}
$$

$ L''_{d} $：日付$d$における1 日当たりの太陽熱補正給湯熱負荷(MJ/d)  
$ L''_{k,d} $：日付$d$における1 日当たりの台所水栓における太陽熱補正給湯熱負荷(MJ/d)  
$ L''_{s,d} $：日付$d$における1 日当たりの浴室シャワー水栓における太陽熱補正給湯熱負荷(MJ/d)  
$ L''_{w,d} $：日付$d$における1 日当たりの洗面水栓における太陽熱補正給湯熱負荷(MJ/d)  
$ L''_{b1,d} $：日付$d$における1 日当たりの浴槽水栓湯はり時における太陽熱補正給湯熱負荷(MJ/d)  
$ L''_{b2,d} $：日付$d$における1 日当たりの浴槽自動湯はり時における太陽熱補正給湯熱負荷(MJ/d)  
$ L''_{ba1,d} $：日付$d$における1 日当たりの浴槽水栓さし湯時における太陽熱補正給湯熱負荷(MJ/d)  
$ L''_{ba2,d} $：日付$d$における1 日当たりの浴槽追焚時における太陽熱補正給湯熱負荷(MJ/d)  

In [ ]:
# 1日当たりの太陽熱補正給湯熱負荷 (5)
def get_L_dashdash_d(L_dashdash_k_d, L_dashdash_s_d, L_dashdash_w_d, L_dashdash_b1_d, L_dashdash_b2_d, L_dashdash_ba1_d,
                     L_dashdash_ba2_d):
    """
    :param L_dashdash_k_d: 1日当たりの台所水栓における太陽熱補正給湯熱負荷 (MJ/d)
    :param L_dashdash_s_d: 1日当たりの浴室シャワー水栓における太陽熱補正給湯負荷 (MJ/d)
    :param L_dashdash_w_d: 1日当たりの洗面水栓における太陽熱補正給湯負荷 (MJ/d)
    :param L_dashdash_b1_d: 1日当たりの浴槽水栓湯はり時における太陽熱補正給湯負荷 (MJ/d)
    :param L_dashdash_b2_d: 1日当たりの浴槽自動湯はり時における太陽熱補正給湯負荷 (MJ/d)
    :param L_dashdash_ba1_d: 1日当たりの浴槽水栓さし湯時における太陽熱補正給湯負荷 (MJ/d)
    :param L_dashdash_ba2_d: 1日当たりの浴槽追焚時における太陽熱補正給湯負荷 (MJ/d)
    :return: 1日当たりの太陽熱補正給湯熱負荷 (MJ/d)
    """
    return L_dashdash_k_d \
           + L_dashdash_s_d \
           + L_dashdash_w_d \
           + L_dashdash_b1_d \
           + L_dashdash_b2_d \
           + L_dashdash_ba1_d \
           + L_dashdash_ba2_d

$$\large
L''_{k,d} = \sum_{t=0}^{23} L''_{k,d,t}
$$

$$\large
L''_{s,d} = \sum_{t=0}^{23} L''_{s,d,t}
$$

$$\large
L''_{w,d} = \sum_{t=0}^{23} L''_{w,d,t}
$$

$$\large
L''_{b1,d} = \sum_{t=0}^{23} L''_{b1,d,t}
$$

$$\large
L''_{b2,d} = \sum_{t=0}^{23} L''_{b2,d,t}
$$

$$\large
L''_{ba1,d} = \sum_{t=0}^{23} L''_{ba1,d,t}
$$

$$\large
L''_{ba2,d} = \sum_{t=0}^{23} L''_{ba2,d,t}
$$

$L''_{k,d}$：：日付$d$における1 日当たりの台所水栓における太陽熱補正給湯熱負荷(MJ/d)  
$L''_{s,d}$：：日付$d$における1 日当たりの浴室シャワー水栓における太陽熱補正給湯熱負荷(MJ/d)  
$L''_{w,d}$：：日付$d$における1 日当たりの洗面水栓における太陽熱補正給湯熱負荷(MJ/d)  
$L''_{b1,d}$：：日付$d$における1 日当たりの浴槽水栓湯はり時における太陽熱補正給湯熱負荷(MJ/d)  
$L''_{b2,d}$：：日付$d$における1 日当たりの浴槽自動湯はり時における太陽熱補正給湯熱負荷(MJ/d)  
$L''_{ba1,d}$：日付$d$における1 日当たりの浴槽水栓さし湯時における太陽熱補正給湯熱負荷(MJ/d)  
$L''_{ba2,d}$：日付$d$における1 日当たりの浴槽追焚時における太陽熱補正給湯熱負荷(MJ/d)  
$ L''_{k,d,t} $：日付$d$の時刻$t$における 1時間当たりの台所水栓における太陽熱補正給湯熱負荷(MJ/h)  
$ L''_{s,d,t} $：日付$d$の時刻$t$における 1時間当たりの浴室シャワー水栓における太陽熱補正給湯熱負荷(MJ/h)  
$ L''_{w,d,t} $：日付$d$の時刻$t$における 1時間当たりの洗面水栓における太陽熱補正給湯熱負荷(MJ/h)  
$ L''_{b1,d,t} $：日付$d$の時刻$t$における 1時間当たりの浴槽水栓湯はり時における太陽熱補正給湯熱負荷(MJ/h)  
$ L''_{b2,d,t} $：日付$d$の時刻$t$における 1時間当たりの浴槽自動湯はり時における太陽熱補正給湯熱負荷(MJ/h)  
$ L''_{ba1,d,t} $：日付$d$の時刻$t$における 1時間当たりの浴槽水栓さし湯時における太陽熱補正給湯熱負荷(MJ/h)  
$ L''_{ba2,d,t} $：日付$d$の時刻$t$における 1時間当たりの浴槽追焚時における太陽熱補正給湯熱負荷(MJ/h)  

In [ ]:
# 1日当たりの台所水栓における太陽熱補正給湯熱負荷 (MJ/d)
def get_L_dashdash_k_d(L_dashdash_k_d_t):
    """
    :param L_dashdash_k_d_t: 1時間当たりの台所水栓における太陽熱補正給湯熱負荷 (MJ/h)
    :return: 1日当たりの台所水栓における太陽熱補正給湯熱負荷 (MJ/d)
    """
    return np.sum(L_dashdash_k_d_t.reshape((365, 24)), axis=1)


# 1日当たりの浴室シャワー水栓における太陽熱補正給湯負荷 (MJ/d)
def get_L_dashdash_s_d(L_dashdash_s_d_t):
    """
    :param L_dashdash_s_d_t: 1時間当たりの浴室シャワー水栓における太陽熱補正給湯負荷 (MJ/h)
    :return: 1日当たりの浴室シャワー水栓における太陽熱補正給湯負荷 (MJ/d)
    """
    return np.sum(L_dashdash_s_d_t.reshape((365, 24)), axis=1)


# 1日当たりの洗面水栓における太陽熱補正給湯負荷 (MJ/d)
def get_L_dashdash_w_d(L_dashdash_w_d_t):
    """
    :param L_dashdash_w_d_t: 1時間当たりの洗面水栓における太陽熱補正給湯負荷 (MJ/h)
    :return: 1日当たりの洗面水栓における太陽熱補正給湯負荷 (MJ/d)
    """
    return np.sum(L_dashdash_w_d_t.reshape((365, 24)), axis=1)


# 1日当たりの浴槽水栓湯はり時における太陽熱補正給湯負荷 (MJ/d)
def get_L_dashdash_b1_d(L_dashdash_b1_d_t):
    """
    :param L_dashdash_b1_d_t: 1時間当たりの浴槽水栓湯はり時における太陽熱補正給湯負荷 (MJ/h)
    :return: 1日当たりの浴槽水栓湯はり時における太陽熱補正給湯負荷 (MJ/d)
    """
    return np.sum(L_dashdash_b1_d_t.reshape((365, 24)), axis=1)


# 1日当たりの浴槽自動湯はり時における太陽熱補正給湯負荷 (MJ/d)
def get_L_dashdash_b2_d(L_dashdash_b2_d_t):
    """
    :param L_dashdash_b2_d_t: 1時間当たりの浴槽自動湯はり時における太陽熱補正給湯負荷 (MJ/h)
    :return: 1日当たりの浴槽自動湯はり時における太陽熱補正給湯負荷 (MJ/d)
    """
    return np.sum(L_dashdash_b2_d_t.reshape((365, 24)), axis=1)


# 1日当たりの浴槽水栓さし湯時における太陽熱補正給湯負荷 (MJ/d)
def get_L_dashdash_ba1_d(L_dashdash_ba1_d_t):
    """
    :param L_dashdash_ba1_d_t: 1時間当たりの浴槽水栓さし湯時における太陽熱補正給湯負荷 (MJ/h)
    :return: 1日当たりの浴槽水栓さし湯時における太陽熱補正給湯負荷 (MJ/d)
    """
    return np.sum(L_dashdash_ba1_d_t.reshape((365, 24)), axis=1)


# 1日当たりの浴槽追焚時における太陽熱補正給湯負荷 (MJ/d)
def get_L_dashdash_ba2_d(L_dashdash_ba2_d_t):
    """
    :param L_dashdash_ba2_d_t: 1時間当たりの浴槽追焚時における太陽熱補正給湯負荷 (MJ/h)
    :return: 1日当たりの浴槽追焚時における太陽熱補正給湯負荷 (MJ/d)
    """
    return np.sum(L_dashdash_ba2_d_t.reshape((365, 24)), axis=1)